# Demo: HRES T0

## Setup

To run this notebook, you will need a recent version of JupyterLab with iPyLeaflet and CartoPy installed. It is recommended to make a new Conda environment and install all three with `conda install -c conda-forge jupyterlab ipyleaflet cartopy`. In addition, you will need the following PyPI packages:

In [ ]:
!pip install -q fsspec gcsfs "zarr<3"

In [ ]:
import logging
import warnings

# In this demo, we silence all warnings.
warnings.filterwarnings("ignore")

# But we do want to show what's happening under the hood!
logging.basicConfig(level=logging.WARNING, format="%(asctime)s [%(levelname)s] %(message)s")
logging.getLogger("aurora").setLevel(logging.INFO)

## Load an Initial Condition

To run Aurora, you need to initialise the model with the state of the world at a particular point in time. For this demo, we initialise Aurora with HRES T0, which is a state-of-the-art data set that is available in real time. Specifically, we initialise the model with the state of the world at 11 May 2022 UTC 12.

In [ ]:
from datetime import datetime

from aurora.foundry.demo.hres_t0_data import load_batch

initial_condition = load_batch(day=datetime(2022, 5, 11))

## Produce Prediction on Azure AI Foundry

Now that we have an initial condition, we connect to the Foundry endpoint and run the model for 4 6-hour steps, producing predictions up to and including UTC 12 the next day.

To run on Foundry, the environment variables `FOUNDRY_ENDPOINT`, `FOUNDRY_TOKEN`, and `BLOB_URL_WITH_SAS` need to be set. If you're unsure on how to set environment variables, see [here](intro.md), or simply copy paste your keys below (but be sure not to commit any keys)! 

In [ ]:
import os

from aurora.foundry import BlobStorageChannel, FoundryClient, submit

foundry_client = FoundryClient(
    endpoint=os.environ["FOUNDRY_ENDPOINT"],
    token=os.environ["FOUNDRY_TOKEN"],
)
channel = BlobStorageChannel(os.environ["BLOB_URL_WITH_SAS"])

predictions = list(
    submit(
        initial_condition,
        model_name="aurora-0.25-finetuned",
        num_steps=4,  # Every step predicts 6 hours into the future.
        foundry_client=foundry_client,
        channel=channel,
    )
)

## Visualisation of Predictions

These are the predictions for temperature globally:

In [ ]:
from datetime import timedelta

import matplotlib.pyplot as plt


def plot(ax, data, time, ic=True):
    ax.imshow(data.numpy() - 273.15, vmin=-50, vmax=50, cmap="RdBu_r")
    ax.set_title(f"2T at {time} " + ("(initial condition)" if ic else "(prediction)"))
    ax.set_xticks([])
    ax.set_yticks([])


fig, axs = plt.subplots(3, 2, figsize=(10, 8))
ic = initial_condition
plot(axs[0, 0], ic.surf_vars["2t"][0, 0], ic.metadata.time[0] - timedelta(hours=6))
plot(axs[0, 1], ic.surf_vars["2t"][0, 1], ic.metadata.time[0])
for i in range(2, 6):
    pred = predictions[i - 2]
    plot(axs[i // 2, i % 2], pred.surf_vars["2t"][0, 0], pred.metadata.time[0], ic=False)
plt.tight_layout()
plt.subplots_adjust(hspace=0.2)

Aurora, however, predicts much more than just temperature: the model predicts the entire state of the atmosphere! Here is an interactive plot of the predictions for wind speed, temperature, and the mean pressure at sea level in the first step of the prediction:

In [ ]:
from aurora.foundry.demo.interactive import interactive_plot

interactive_plot(predictions[0], width="1000px", height="500px")